In [ ]:
# ! conda install -c conda-forge imbalanced-learn -y
# ! pip install nose
# ! pip install imbalanced-ensemble           
# ! pip install threadpoolctl

In [3]:
import re

def read_dot_dat_file(path):
    datContent = [i.strip().split() for i in open(path).readlines()]
    r = re.compile("@inputs.*")
    _at_data = datContent.index(['@data'])
    assert datContent[0][0] == '@relation'
    assert datContent[_at_data-1][0] == '@outputs'
    assert datContent[_at_data-2][0] == '@inputs'
    print(datContent[_at_data-3][2:])
    assert len(datContent[_at_data-3][2:]) == 2   # Two Class

    col_names = datContent[_at_data-2][1:]
    col_names.append(datContent[_at_data-1][1])
    
    df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=r', ', engine='python')
    # df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=", ", engine='python')

    class1 = datContent[_at_data-3][2:][0].replace("{","").replace(",","")
    class2 = datContent[_at_data-3][2:][1].replace("}","").replace(",","")

    df['Class'] = df['Class'].replace({class1: 1, class2: -1})
    
    return df



# temp

In [3]:
# import pandas as pd 
# import numpy as np
# import os

# base_path = 'Datasets/'
# # need_to_convert = ['wisconsin-5-fold','new-thyroid2-5-fold','new-thyroid1-5-fold']
# # need_to_convert = ['yeast-1_vs_7-5-fold', 'led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold', 'ecoli-0-3-4-6_vs_5-5-fold', 'abalone19-5-', 'abalone19-5-fold']
# # need_to_convert = ['ecoli-0-1_vs_5-5-fold','ecoli-0-1-4-7_vs_5-6-5-fold','ecoli-0-3-4-7_vs_5-6-5-fold','glass-0-1-4-6_vs_2-5-fold', 'ecoli-0-4-6_vs_5-5-fold',
# #                   'yeast-0-3-5-9_vs_7-8-5-fold','yeast-1-2-8-9_vs_7-5-fold']
# need_to_convert = ['yeast-1-2-8-9_vs_7-5-fold']
# for needed in need_to_convert:
#     for datFile in glob.glob(base_path+needed+'/*.dat'):
#         print(datFile)
#         df = read_dot_dat_file(datFile)
#         df.to_excel(base_path+needed+'/'+datFile.split('/')[-1].split('.')[0]+'.xlsx')


In [1]:
import time
import os
import glob
import numpy as np
import random  
import pandas as pd 
import random

from sklearn.tree import DecisionTreeClassifier 

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc,f1_score, matthews_corrcoef, precision_score, recall_score
from imblearn.metrics import geometric_mean_score


from imblearn.ensemble import RUSBoostClassifier
from imbalanced_ensemble.ensemble import SMOTEBoostClassifier
from maatpy.classifiers import AdaCost

from OUBoost import OUBoost

base_path = 'Datasets/'
datasets  = os.listdir(base_path) 
classifiers = {"RUS": RUSBoostClassifier(random_state=0, algorithm='SAMME', base_estimator=DecisionTreeClassifier(max_depth=10)),
              "SMOTE": SMOTEBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators = 100, algorithm='SAMME', random_state=0),
              "Ada1": AdaCost(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=100, algorithm='adac1',random_state=0),
              "AdaCost": AdaCost(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=100, algorithm='adacost',random_state=0),
              "OUBoost": OUBoost(learning_rate=0.3, n_samples=100, n_estimators=100)}
            

def clasify(dataset, classifier):
    mcc = []
    f1 = []
    auc_a = []
    gmean = []
    times = []
    y_preds = []
    precisions = []
    recalls = []
    traFiles = sorted(glob.glob(base_path+dataset+'/*tra.xlsx'))
    tstFiles = sorted(glob.glob(base_path+dataset+'/*tst.xlsx'))
    for traPath, tstPath in zip(traFiles, tstFiles):
        print(traPath)
        
        df_train = pd.read_excel(traPath)
        df_test = pd.read_excel(tstPath)

        x_train= df_train.iloc[:, 1:-1]
        y_train = df_train.iloc[:, -1]
        x_test= df_test.iloc[:, 1:-1]
        y_test = df_test.iloc[:, -1]
        
        #####
        # for some dataset get error Unknown label type: 'unknown'
        y_train = y_train.astype('int')
        y_test = y_test.astype('int')
        
        st = time.time()
        clf = classifiers[classifier]

        clf.fit(x_train, y_train)

        y_pred = clf.predict(x_test) 
        et = time.time()
        y_preds.append(y_pred)
        # compute error
        mcc.append(matthews_corrcoef(y_test, y_pred))
        #--------------------------------
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auc_a.append(auc(fpr, tpr))
        #--------------------------------
        f1.append(f1_score(y_test, y_pred))
        #--------------------------------
        gmean.append(geometric_mean_score(y_test, y_pred, labels=[1, -1]))
        
        #time of train and test
        times.append(et - st)
        
        precisions.append(precision_score(y_test, y_pred))
        recalls.append(precision_score(y_test, y_pred))
        
    return {"precision": precisions, "recall": recalls, "mcc": mcc, "auc": auc_a, "f1": f1, "gmean": gmean, "exe_time": times, "y_pred": y_pred}

# aaaaaa =  clasify(datasets[0], 'Ada1')
# aaaaaa =  clasify('yeast-1-2-8-9_vs_7-5-fold', 'RUS')
# aaaaaa
clasify('yeast4-5-fold', 'OUBoost')

Datasets/yeast4-5-fold/yeast4-5-1tra.xlsx
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 1146 (1147, 7)
indexs.max(), X_maj.shape 32 (1147, 7)
indexs.max(), X_maj.shape 1339 (1147, 7)


IndexError: index 1269 is out of bounds for axis 0 with size 1147

In [89]:
# # #temp 
# for i in range(len(datasets)):
#     if not os.path.isdir('./Results/'+datasets[i]):
#         os.mkdir('./Results/'+datasets[i])
#     for classifier in classifiers:
#         if not os.path.isdir('./Results/'+datasets[i]+'/'+classifier):
#             os.mkdir('./Results/'+datasets[i]+'/'+classifier)
            


In [5]:
def res_to_files(dataset, classifier, dict_res):
    np.save('./Results/'+dataset+'/'+classifier+'/mcc.npy',dict_res['mcc'])
    np.save('./Results/'+dataset+'/'+classifier+'/auc.npy',dict_res['auc'])
    np.save('./Results/'+dataset+'/'+classifier+'/f1.npy',dict_res['f1'])
    np.save('./Results/'+dataset+'/'+classifier+'/gmean.npy',dict_res['gmean'])
    np.save('./Results/'+dataset+'/'+classifier+'/exe_time.npy',dict_res['exe_time'])
    np.save('./Results/'+dataset+'/'+classifier+'/y_pred.npy',dict_res['y_pred'])
    

In [37]:
# datasets

In [ ]:
# for classifier in classifiers:
#     for dataset in datasets:
#         if not os.path.exists('./Results/'+dataset+'/'+classifier+'/y_pred.npy'):
#             print(classifier," => ", dataset, "=>", end=' ')
#             # tmp_res = clasify(dataset, classifier)
#             # print(tmp_res['auc'])
#             try:
#                 tmp_res = clasify(dataset, classifier)
#                 res_to_files(dataset, classifier, tmp_res)
#             except:
#                 print(dataset, classifier)

classifier = 'Ada1'
for dataset in datasets:
    if not os.path.exists('./Results/'+dataset+'/'+classifier+'/y_pred.npy'):
        print(classifier," => ", dataset, "=>", end=' ')
        # try:
        tmp_res = clasify(dataset, classifier)
        res_to_files(dataset, classifier, tmp_res)
        # except:
            # print(dataset, classifier)

# classifier = 'AdaCost'
# dataset = 'abalone19-5-'
# if not os.path.exists('./Results/'+dataset+'/'+classifier+'/y_pred.npy'):
#     print(classifier," => ", dataset, "=>", end=' ')
#     tmp_res = clasify(dataset, classifier)
#     res_to_files(dataset, classifier, tmp_res)
#     print(tmp_res['auc'])

        

In [ ]:
troubs = {}
for classifier in classifiers:
    for dataset in datasets:
        files = glob.glob('./Results/'+dataset+'/'+classifier+'/*.npy')
        if len(files) != 6:
            if dataset in troubs:
                troubs[dataset].append(classifier)
            else:
                troubs[dataset] = [classifier]
            print(dataset, classifier)
        
# print([dataset for dataset in troubs if len(troubs[dataset])==4])

In [12]:
# # convert categorical variables into numerical
# from sklearn.preprocessing import LabelEncoder
 
# le = LabelEncoder()
 
# # Using .fit_transform function to fit label
# # encoder and return encoded label

# base_path = 'Datasets/'
# need_to_convert = ['abalone19-5-', 'abalone19-5-fold']
# for needed in need_to_convert[1:]:
#     for datFile in glob.glob(base_path+needed+'/*.dat'):
#         print(datFile)
#         df = read_dot_dat_file(datFile)
#         label = le.fit_transform(df['Sex,'])        
#         df['Sex,'] = label
#         # df.to_excel(base_path+needed+'/'+datFile.split('/')[-1].split('.')[0]+'.xlsx')
#         break
        

In [11]:
# df = read_dot_dat_file(glob.glob(base_path+datasets[0]+'/*.dat')[0])
# df.to_excel('temp.xlsx')

In [10]:
# pd.read_excel('temp.xlsx').iloc[:, 0:-1]

In [ ]:
# yeast-1-2-8-9_vs_7-5-fold RUS
# shuttle-c2-vs-c4-5-fold SMOTE
# ecoli-0-1-3-7_vs_2-6-5-fold SMOTE
# yeast4-5-fold Ada1
# abalone19-5- Ada1
# abalone19-5-fold Ada1
# abalone19-5- AdaCost
# abalone19-5-fold AdaCost